# Explore here

In [2]:
# Your code here
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras import Input, Model
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Rescaling
import requests
from zipfile import ZipFile
from io import BytesIO
import os
import shutil
import tensorflow as tf

In [3]:
#  Download the dataset
url = "https://storage.googleapis.com/datascience-materials/dogs-vs-cats.zip"
response = requests.get(url)

#  Extract to "dogs-vs-cats"
with ZipFile(BytesIO(response.content)) as zip_file:
    zip_file.extractall("dogs-vs-cats")

#  Correct the nested structure
# Original images are in: dogs-vs-cats/dogs-vs-cats/train/
source_train = "dogs-vs-cats/dogs-vs-cats/train"
dest_train = "dogs-vs-cats/train"

os.makedirs(dest_train, exist_ok=True)

# Move all images into a single folder first
for filename in os.listdir(source_train):
    src_path = os.path.join(source_train, filename)
    dst_path = os.path.join(dest_train, filename)
    shutil.move(src_path, dst_path)

#  Create cats/ and dogs/ folders for TensorFlow
cats_folder = os.path.join(dest_train, "cats")
dogs_folder = os.path.join(dest_train, "dogs")
os.makedirs(cats_folder, exist_ok=True)
os.makedirs(dogs_folder, exist_ok=True)

for filename in os.listdir(dest_train):
    file_path = os.path.join(dest_train, filename)
    if filename.startswith("cat"):
        shutil.move(file_path, cats_folder)
    elif filename.startswith("dog"):
        shutil.move(file_path, dogs_folder)

#  Load the dataset
train_dataset = tf.keras.utils.image_dataset_from_directory(
    dest_train,
    labels="inferred",
    label_mode="binary",
    batch_size=32,
    image_size=(150,150),
    shuffle=True
)

print("Dataset loaded successfully!")



Found 25000 files belonging to 2 classes.


2026-01-14 18:20:21.888805: E external/local_xla/xla/stream_executor/cuda/cuda_platform.cc:51] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)


Dataset loaded successfully!


In [4]:
import random
import shutil
import os

os.makedirs("dogs-vs-cats/test/cats", exist_ok=True)
os.makedirs("dogs-vs-cats/test/dogs", exist_ok=True)

train_cats = os.listdir("dogs-vs-cats/train/cats")
train_dogs = os.listdir("dogs-vs-cats/train/dogs")

for f in random.sample(train_cats, 500):
    shutil.move(f"dogs-vs-cats/train/cats/{f}","dogs-vs-cats/test/cats/{f}")
for f in random.sample(train_dogs, 500):
    shutil.move(f"dogs-vs-cats/train/dogs/{f}","dogs-vs-cats/test/dogs/{f}")

In [5]:
datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)

train_data = datagen.flow_from_directory(
    "dogs-vs-cats/train",
    target_size=(150,150),
    batch_size=32,
    class_mode='binary',
    subset='training'
)

val_data = datagen.flow_from_directory(
    "dogs-vs-cats/train",
    target_size=(150,150),
    batch_size=32,
    class_mode='binary',
    subset='validation'
)


Found 19200 images belonging to 2 classes.
Found 4800 images belonging to 2 classes.


In [10]:
from tensorflow import keras
from tensorflow.keras import layers

In [11]:
inputs = keras.Input(shape=(150, 150, 3)) #rgb images
x = layers.Conv2D(64, 3, activation='relu')(inputs)
x = layers.MaxPooling2D()(x)
x = layers.Conv2D(128, 3, activation='relu')(x)
x = layers.MaxPooling2D()(x)
x = layers.Conv2D(256, 3, activation='relu')(x)
x = layers.GlobalAveragePooling2D()(x)
outputs = layers.Dense(1, activation='sigmoid')(x)
model = keras.Model(inputs, outputs)

In [12]:
model.compile(optimizer='adam',
              loss ='binary_crossentropy',
              metrics=['accuracy'])

In [13]:
model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 150, 150, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d (Conv2D)                 │ (None, 148, 148, 64)   │         1,792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 74, 74, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 72, 72, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 36, 36, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 34, 34, 256)    │       295,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d        │ (None, 256)            │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 1)              │           257 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 371,073 (1.42 MB)

 Trainable params: 371,073 (1.42 MB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
model.fit(train_data,
          validation_data=val_data,
          epochs=5,
          verbose=2,
          steps_per_epoch=100,
          validation_steps=50)

In [ ]:
loss, accuracy = model.evaluate(val_data, verbose=2)
print(f"Validation Accuracy: {accuracy}")